In [45]:
from Bio import Entrez
from Bio import Medline
import datetime 

Entrez.email = "bgreshake@googlemail.com"

search_term = "((gender[Title/Abstract]) OR sex[Title/Abstract]) AND (fMRI[Title/Abstract] OR EEG[Title/Abstract])"

In [46]:
crawltime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# get number of articles we need to fetch
handle = Entrez.esearch(db="pubmed", term=search_term)
record = Entrez.read(handle)
article_number = record["Count"]
handle.close()

In [47]:
# get all articles from pubmed
handle = Entrez.esearch(db="pubmed", term=search_term, retmax=article_number)
record = Entrez.read(handle)
id_list = record["IdList"]
handle.close()

In [48]:
# split input list in chunks of 150 so pubmed will honor our requests

id_list_chunks = [id_list[x:x+150] for x in range(0, len(id_list), 150)]
len(id_list_chunks)
print len(id_list_chunks[-1])

3


In [49]:
all_records = []
for i in id_list_chunks:
    handle = Entrez.efetch(db="pubmed", id=i, rettype="medline",
                           retmode="text")
    records = Medline.parse(handle)
    records = list(records)
    all_records += records
    handle.close()

In [53]:
outfile = "../data/%s-pubmed-crawl.csv" % crawltime
output = open(outfile,"w")
output.write("#search term: %s \n" % search_term)
output.write("pubmed_ID\tDOI\n")

no_doi_outfile = "../data/%s-pubmed-crawl-no-doi.csv" % crawltime
no_doi_output = open(no_doi_outfile,"w")
no_doi_output.write("#search term: %s \n" % search_term)
no_doi_output.write("pubmed_ID\n")


for record in all_records:
    doi = "-"
    if "AID" in record.keys():
        for aid in record["AID"]:
            if "[doi]" in aid:
                doi = aid.replace("[doi]","")
    
    if doi != "-":
        output.write(record["PMID"] + "\t" + doi + "\n")        
    else:
        no_doi_output.write(record["PMID"] + "\n")        
output.close()
no_doi_output.close()

In [44]:
datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

'2017-09-04-18-40-28'